In [ ]:
#Import ee and required packages
import ee
ee.Initialize()
import pandas as pd 
import numpy as np
import glob
import geopandas as gpd

In [ ]:
# VARIABLE DECLARATIONS

STATE="Montana"



state_abbrevs = {
    'Montana' : 'MT'
}

In [ ]:
# TESTING AREA
print(STATE)
print(state_abbrevs[STATE])

In [ ]:
#If you need to create the spatially thinned asset...Otherwise skip to Define Modular Variables below
#Define GEE asset/location of desired dataset (Formatted CSV must be uploaded to your GEE assets with Lat/Long columns defined 
#before starting)
Taxa_og = ee.FeatureCollection('users/kjchristensen93/EBT_data/EBT_mfish_data_presence_heuristic')
coll = ee.FeatureCollection(Taxa_og) 
distance = 500

In [ ]:
# Spatially thin locations and export to asset
#//========================================================

def filterDistance(points, distance):
    ## EE FUNCTION THAT TAKES A FEATURE COLLECTION AND FILTERS 
    ## BY A MINIMUM DISTANCE IN METERS
    def iter_func(el, ini):
        ini = ee.List(ini)
        fcini = ee.FeatureCollection(ini)
        buf = ee.Feature(el).geometry().buffer(distance)
        s = fcini.filterBounds(buf).size()
        cond = s.lte(0)
        return ee.Algorithms.If(cond, ini.add(el), ini)
    filt2 = ee.List([])
    filt = points.iterate(iter_func, filt2)
    filtered = ee.FeatureCollection(ee.List(filt))
    return filtered

#//========================================================

In [ ]:
# Function that filters the feature collection and spatially thins it
def filter_date_space(date):
    start_date = ee.Date(date).advance(1, 'year')
    end_date = start_date.advance(1, 'year')
    points_in_that_year = coll.filterDate(start_date, end_date)
    
    spatially_filt = filterDistance(points_in_that_year, distance)
    
    return spatially_filt

In [ ]:
# Spatially thin locations and export to asset
# Performs the spatial thinning algorithm on each year separately
feats = s_dates.map(filter_date_space)
def merge_coll(first_year, second_year):
    return ee.FeatureCollection(first_year).merge(ee.FeatureCollection(second_year))

# Combine each of the resultant filtered collections
first = ee.FeatureCollection(Taxa_og)
spatially_thin = ee.FeatureCollection(feats.iterate(merge_coll, first))

In [ ]:
export3 = ee.batch.Export.table.toAsset(collection = spatially_thin,
                    description = 'EBT_SThin', # n<-------- CHANGE NAME FOR DIFFERENT DATA
                    assetId = 'users/kjchristensen93/EBT_SThin') # <----- CHANGE Export location FOR DIFFERENT USER

export3.start()

In [ ]:
#Define Modular Variables:

#If you have a spatially thinned data set, start here after initializing ee

#Taxa thinned dataset
SThin = ee.FeatureCollection('users/mstokowski/EBT_SThin')
#Study dates
#Note we are limited to 2002 - 2018 due to the water year covariate 
s_dates = ee.List ([ 
ee.Date('2002-01-01'),
ee.Date('2003-01-01'),ee.Date('2004-01-01'),
ee.Date('2005-01-01'),ee.Date('2006-01-01'),
ee.Date('2007-01-01'),ee.Date('2008-01-01'),
ee.Date('2009-01-01'),ee.Date('2010-01-01'),
ee.Date('2011-01-01'),ee.Date('2012-01-01'),
ee.Date('2013-01-01'),ee.Date('2014-01-01'),
ee.Date('2015-01-01'),ee.Date('2016-01-01'),
ee.Date('2017-01-01'),ee.Date('2018-01-01')]) 

#HUC state geojson file 
HUC_state = ('C:/Users/YOURFOLDER/YOUR_STATE_HUC.geojson')
#Define export locations:
#GEE yearly covariate folder
assetId = ('users/kjchristensen93/covariates_test') 
#User training csv local directory folder
trainingdata = ('/Users/myles/Documents/flbs')
#User decadal image local directory folder
decadalfolder = ('C:/Users/YOUR_DECADE_FOLDER_PATH')
#Define export naming convention? Maybe we define a function within code above for naming conventions


#### ML Variables ####

#Training Glob
trainingglob = ('/Users/myles/Documents/*.csv')
# trainingglob = ((trainingdata)/*.csv) will this work?
#decadal CSV directory and naming conventions
decade1 = ('/Users/myles/Documents/flbs/decade1_filename.csv')
decade2 =('/Users/myles/Documents/flbs/decade2.filename.csv')
#decadal predictions
decade1_pred = ('/Users/myles/Documents/flbs/decade1_pred_filename.csv')
decade2_pred = ('/Users/myles/Documents/flbs/decade2_pred_filename.csv')

#######################

In [ ]:
#This list dictates what years will be exported for both the Yearly Covariate Images and the Yearly Training CSVS
# can this be changed to a list for intermitent datasets missing years? Empty outputs causes issues later on....
import time
# Enter start year for Y and end year for Y
years = [str(y) for y in list(range(2002, 2005))]


In [ ]:
# Export data using python API magic
# Define geometry by changing state name so we can export the whole state at once
states = ee.FeatureCollection("TIGER/2016/States")
#Enter state 2-digit abbreviation for study area
geometry = states.filter(ee.Filter.eq('NAME',STATE)).geometry()

In [ ]:
# Shape file containing HUC polygons
HUC = ee.FeatureCollection("USGS/WBD/2017/HUC12")
# Choose state to clip HUC by. Change Abbreviation to match dataset 
#Enter state full name for X (i.e., Illinois/ look at dataset for formats for this stuff)
HUC_clip = HUC.filter(ee.Filter.eq('states',state_abbrevs[STATE]))

In [ ]:
#embed observation Year as system:start_time for thinned dataset 
# We have had to add this "Year Column" manually to the datasets.  Make sure your dataset has correct column headings
def embedd_date(x):
    yr = ee.Number(x.get("Year"))
    eedate = ee.Date.fromYMD(yr, 1, 1)
    return x.set("system:time_start", eedate)
SThin_map = SThin.map(embedd_date)

In [ ]:
#Build Big Raster Image
## Import assets
# MODIS Mission
modusGlobal = ee.ImageCollection("MODIS/006/MYD11A2")

# Primary Productivity
GPP = ee.ImageCollection("UMT/NTSG/v2/LANDSAT/GPP")

# Surface water
pikelSurfaceWater = ee.Image("JRC/GSW1_1/GlobalSurfaceWater")

# Elevation
DEM = ee.Image("USGS/NED")

# Enhanced Vegetation Index and NDVI
modusVeg = ee.ImageCollection("MODIS/006/MYD13A2")

# Heat Isolation Load
CHILI = ee.Image("CSP/ERGo/1_0/Global/SRTM_CHILI")

# Topographic Diversity
topoDiversity = ee.Image("CSP/ERGo/1_0/Global/ALOS_topoDiversity")

# Vegetation Continuous Field product - percent tree cover, etc
VCF = ee.ImageCollection("MODIS/006/MOD44B")

# Human Modification index
gHM = ee.ImageCollection("CSP/HM/GlobalHumanModification")

# Climate information
NLDAS = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")

# Shape file containing Country Boundaries
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Shape file containing HUC polygons
HUC = ee.FeatureCollection("USGS/WBD/2017/HUC12")

# Dynamic Surface Water metric
pekel_monthly_water = ee.ImageCollection("JRC/GSW1_2/MonthlyHistory")

# Static surface water metric
pekel_static_water = ee.ImageCollection('JRC/GSW1_2/MonthlyRecurrence')



In [ ]:
## Select features, etc
#========================================================
#Rename Bands and select bands, etc
#========================================================


NLDAS_precip = NLDAS.select("total_precipitation");
NLDAS_temp = NLDAS.select("temperature");
NLDAS_humid = NLDAS.select("specific_humidity");
NLDAS_potEvap = NLDAS.select("potential_evaporation");


CHILI = CHILI.rename(['Heat_Insolation_Load'])
srtmChili = CHILI.select('Heat_Insolation_Load');
topoDiversity = topoDiversity.rename(["Topographic_Diversity"])
topoDiv = topoDiversity.select("Topographic_Diversity")
footprint = ee.Image(gHM.first().select("gHM"));

# Surface water occurrence
sw_occurrence = pekel_static_water\
                      .select('monthly_recurrence')\
                      .mean()\
                      .rename(['SurfaceWaterOccurrence'])\
                      .unmask()


In [ ]:
## Define helper filters and lists to iterate over
#========================================================
# Build Lists from which to map over
#========================================================
# List from which absences will be built
ee_dates = ee.List(s_dates)


# Required for exporting presence locations due to covariate availability 
ee_dates_presence = ee.List(s_dates) ##FIXME: Optimize. We run the same function twice, could clean up


In [ ]:
## Mask features by quality control bands
# ========================================================
# Masking TPP via quality control bands
# ========================================================
def gpp_qc(img):
    img2 = img.rename(['GPP','QC']);
    quality = img2.select("QC")
    mask = quality.neq(11) \
                .And(quality.neq(10)) \
                .And(quality.neq(20)) \
                .And(quality.neq(21)) 
    return img2.mask(mask)

GPP_QC = GPP.map(gpp_qc);

# ========================================================
# Masking LST via quality control bands
# ========================================================
def lst_qc(img):
    quality = img.select("QC_Day")
    mask = quality.bitwiseAnd(3).eq(0) \
                .And(quality.bitwiseAnd(12).eq(0))
    return img.mask(mask)

LST = modusGlobal.map(lst_qc) \
                 .select("LST_Day_1km");

        
# ========================================================
# Mask Modus Vegetation Indices by quality flag
# ========================================================
def modusQC(image):
    quality = image.select("SummaryQA")
    mask = quality.eq(0)
    return image.updateMask(mask)

modusVeg_QC = modusVeg.map(modusQC)
EVI = modusVeg_QC.select("EVI")
NDVI = modusVeg_QC.select("NDVI")

# ========================================================
# Mask Continuous Fields via quality control band
# ========================================================
def VCFqc(img):
    quality = img.select("Quality")
    mask = quality.bitwiseAnd(2).eq(0) \
                    .And(quality.bitwiseAnd(4).eq(0)) \
                    .And(quality.bitwiseAnd(8).eq(0)) \
                    .And(quality.bitwiseAnd(16).eq(0)) \
                    .And(quality.bitwiseAnd(32).eq(0))

    return img.mask(quality)

VCF_qc = VCF.map(VCFqc)


#========================================================
# Define Point Joins such that each HUC contains a list of observational data:
#========================================================
distFilter = ee.Filter.intersects(**{
  'leftField': '.geo', 
  'rightField': '.geo', 
  'maxError': 100
});

pointJoin = ee.Join.saveAll(**{
  'matchesKey': 'Points',
});

In [ ]:
## Annual Cube function
#========================================================
# "Builder Function" -- processes each annual variable into a list of images
#========================================================

def build_annual_cube(d):
    # Set start and end dates for filtering time dependent predictors (SR, NDVI, Phenology)
      # Advance startDate by 1 to begin with to account for water year (below)
    startDate = (ee.Date(d).advance(1.0,'year').millis()) ## FIXME: Why do we advance a year? this give 2003-2019 instead of 2002-2018
    endDate = ee.Date(d).advance(2.0,'year').millis()

  #========================================================
  #Define function to compute seasonal information for a given variable
  #========================================================
    def add_seasonal_info(imgCol,name,bandName):
        winter = imgCol.filterDate(winter_start,winter_end)
        spring = imgCol.filterDate(spring_start,spring_end)
        summer = imgCol.filterDate(summer_start,summer_end)
        fall = imgCol.filterDate(fall_start,fall_end)

        winter_tot = winter.sum()
        spring_tot = spring.sum()
        summer_tot = summer.sum()
        fall_tot = fall.sum()

        winter_max = winter.max()
        winter_min = winter.min()
        spring_max = spring.max()
        spring_min = spring.min()
        summer_max = summer.max()
        summer_min = summer.min()
        fall_max = fall.max()
        fall_min = fall.min()

        winter_diff = winter_max.subtract(winter_min)
        spring_diff = spring_max.subtract(spring_min)
        summer_diff = summer_max.subtract(summer_min)
        fall_diff = fall_max.subtract(fall_min)

        names = ['winter_total'+name,'spring_total'+name,'summer_total'+name,
                      'fall_total'+name]

        return winter_tot.addBands([spring_tot,summer_tot,fall_tot]) \
                         .rename(names)

  # Set up Seasonal dates for precip, seasonal predictors
    winter_start = ee.Date(startDate)
    winter_end = ee.Date(startDate).advance(3,'month')
    spring_start = ee.Date(startDate).advance(3,'month')
    spring_end = ee.Date(startDate).advance(6,'month')
    summer_start = ee.Date(startDate).advance(6,'month')
    summer_end = ee.Date(startDate).advance(9,'month')
    fall_start = ee.Date(startDate).advance(9,'month')
    fall_end = ee.Date(endDate)

  # Aggregate seasonal info for each variable of interest (potEvap neglected purposefully)
    seasonal_precip = add_seasonal_info(NLDAS_precip,"Precip","total_precipitation")
    seasonal_temp = add_seasonal_info(NLDAS_temp,"Temp","temperature")
    seasonal_humid = add_seasonal_info(NLDAS_humid,"Humidity","specific_humidity")

    waterYear_start = ee.Date(startDate).advance(10,'month')
    waterYear_end = waterYear_start.advance(1,'year')

  #========================================================
  # Aggregate Other Covariates
  #========================================================

  # Vegetative Continuous Fields
    meanVCF = VCF.filterDate(startDate, endDate)\
                 .mean()
    
#     VCF_qc.filterDate(startDate, endDate) \
#                       .mean()

  # Filter Precip by water year to get total precip annually

    waterYearTot = NLDAS_precip.filterDate(waterYear_start,waterYear_end) \
                                 .sum()

  # Find mean EVI per year:
    maxEVI = EVI.filterDate(startDate,endDate) \
                  .mean() \
                  .rename(['Mean_EVI'])

  #Find mean NDVI per year:
    maxNDVI = NDVI.filterDate(startDate,endDate) \
                    .mean() \
                    .rename(["Mean_NDVI"])

  # Find flashiness per year by taking a Per-pixel Standard Deviation:
    flashiness_yearly = ee.Image(pekel_monthly_water.filterDate(startDate,endDate) \
                                                      .reduce(ee.Reducer.sampleStdDev()) \
                                                      .select(["water_stdDev"])) \
                                                      .rename("Flashiness")

  # Find max LST per year:
    maxLST = LST.max().rename(["Max_LST_Annual"])

  # Find mean GPP per year:
    maxGPP = GPP_QC.filterDate(startDate,endDate) \
                      .mean() \
                      .rename(['Mean_GPP','QC'])

  # All banded images that don't change over time
    static_input_bands = sw_occurrence.addBands(DEM.select("elevation")) \
                                          .addBands(srtmChili) \
                                          .addBands(topoDiv) \
                                          .addBands(footprint)

  # Construct huge banded image
    banded_image = static_input_bands \
                          .addBands(srcImg = maxLST, names = ["Max_LST_Annual"]) \
                          .addBands(srcImg = maxGPP, names = ["Mean_GPP"]) \
                          .addBands(srcImg =  maxNDVI, names = ["Mean_NDVI"]) \
                          .addBands(srcImg = maxEVI, names = ["Mean_EVI"]) \
                          .addBands(meanVCF.select("Percent_Tree_Cover")) \
                          .addBands(seasonal_precip) \
                          .addBands(flashiness_yearly) \
                          .set("system:time_start",startDate)

    return banded_image.unmask()




In [ ]:
#========================================================
# REDUCE REGIONS FUNCTION
#========================================================
def reduce_HUCS(img):
    # Cast to image because EE finicky reasons
    img = ee.Image(img)
    startDate = ee.Date(img.get("system:time_start"))
    endDate = startDate.advance(1,'year')
    
    # Find each occurrence record within the year
    pointsInThatYear = SThin_map.filterDate(startDate, endDate);
    
    # Apply spatial join to presence data 
    new_HUCS = pointJoin.apply(HUC_clip,pointsInThatYear,distFilter);
    
    #Reduce the Multi-band image to HUC-level aggregates
    reduced_image = img.reduceRegions(**{
                              'collection': new_HUCS,
                              'reducer': ee.Reducer.mean(),
                              'crs': 'EPSG:4326',
                              'scale': 100,
                              'tileScale': 16}).map(lambda y:y.set({'Time': img.get("system:time_start")}))
    
    def getAvgPresence (feat):
        pts = ee.List(feat
              .get('Points')) \
              .map(lambda pt: ee.Feature(pt).get('Present'))
                          
        avg = ee.List(pts).reduce(ee.Reducer.mean())
   
        return feat.set("Avg_Presence", avg)
    
    return reduced_image.map(getAvgPresence)

In [ ]:

#========================================================
# Run covariate algorithm and build a list of images
# with each image corresponding to each year and each band corresponding to each covariate
#========================================================

# Image Collection
banded_images = ee.ImageCollection(ee_dates.map(build_annual_cube))

# List form
banded_images_list = ee.List(ee_dates.map(build_annual_cube))

annual_stacks = ee.FeatureCollection(banded_images.map(reduce_HUCS))




In [ ]:
#Skip this step if you already have them stored in GEE
#Export Yearly Covariate Images

# Export each image within the for loop
for i,y in zip(range(len(years)), years):
    print("Starting", y)
    img = ee.Image(ee.List(banded_images_list).get(ee.Number(i)))
    export = ee.batch.Export.image.toAsset(image = img,
                    description = 'covariate_'+y,
                    assetId = (assetId) +y, 
                    region = ee.Geometry(geometry),
                    scale =  100,
                    maxPixels = 1e13)
    export.start()
    
    print(y,"status:    ", export.status()['state'])

    # Wait for 30 seconds so that the export['state'] gives insightful information
    time.sleep(15)
    print(y,"status:    ", export.status()['state'])
    
    
    # If this status is "RUNNING", then there are no egretious syntax errors. 
    # However, it is still possible that these export commands fail after more than 30 seconds.
    # In that case, it is likely that there is a Computation Time Out Error (remember exporting the annual stacks)
    time.sleep(15)
    print(y,"status:    ", export.status()['state'])
    

In [ ]:
#Start Here if you have yearly covariates created

#Export training CSVs
## Reduce Regions from existing images

#Can we set this up such that this is automatically defined when we define the year range above?

# COVARIATE IMAGES  

path = 'users/kjchristensen93/covariates_test'
years = range(2002, 2005)
images = map(lambda x: ee.Image(path + str(x)), years)
banded_images_asset_list = ee.List(list(images))

for i in range(len(years)):
    print("Starting", i)
    
    img = ee.Image(banded_images_asset_list.get(i))
    data = reduce_HUCS(img) 
    
    ## PYTHON API MAGIC!! LOOK HERE
    my_csv = pd.DataFrame([x['properties'] for x in data.getInfo()['features']])
    
    # From there, we can write it directly to our directory and stitch it together afterwards
    my_csv.to_csv((trainingdata) + str(2002+i) + '.csv', index=False) 
    print("Finished", i)

In [ ]:
#export the information that we will use to project habitat suitability. 
#Decades were convenient for RBT, but not other taxa with less data/ we can change.
# Change to match dataset

#Can we set this up such that this is automatically defined when we define the year range above?
first_decade = ee.ImageCollection.fromImages([image1, 
                                              image2, 
                                              #image3, 
                                              #image4,
                                              #image5, 
                                              #image6, 
                                              #image7, 
                                              #image8
                                                ]).mean()

#second_decade = ee.ImageCollection.fromImages([image9, 
                                               #image10, 
                                               #image11, 
                                               #image12, 
                                               #image13, 
                                               #image14, 
                                               #image15
                                                  #]).mean()


In [ ]:
# Export these data as csvs
first_decade_img = ee.Image(first_decade)

first_csv = first_decade_img.reduceRegions(**{
                              'collection': HUC_clip,
                              'reducer': ee.Reducer.mean(),
                              'crs': 'EPSG:4326',
                              'scale': 100,
                              'tileScale': 16})

#PYTHON API MAGIC!! LOOK HERE
first_decade_data = pd.DataFrame([x['properties'] for x in first_csv.getInfo()['features']])

# From there, we can write it directly to our directory and stitch it together afterwards
#maybe we should think about 2 and 5 year bins due to limitations of datasets for some taxa/ to make more useful for managers
first_decade_data.to_csv(decade1)



In [ ]:
# Combined data

dfs = []
for file in glob.glob(trainingglob):
    dfs.append(pd.read_csv(file))

df = pd.DataFrame(pd.concat(dfs, ignore_index=True))

In [ ]:
df

In [ ]:
to_drop = ['Points', 'areaacres', 'areasqkm', 
           'gnis_id','huc12', 'humod','hutype',
           'loaddate', 'metasource', 'name', 
           'noncontr00','noncontrib','shape_area',
           'shape_leng', 'sourcedata','sourcefeat',
           'sourceorig','states','tnmid', 'tohuc', 'Time']

df['Avg_Presence'] = [1 if x > 0 else 0 for x in df['Avg_Presence']]
df.drop(columns = to_drop, inplace=True)

In [ ]:
# Sanity Check
df.head()

In [ ]:
## Train Models

#install skLearn if you don't already have it

# Ensemble modeling class
from sklearn.metrics import *
class Ensemble():
    def __init__(self, models = []):
        self.models = models
        self.accs_dict = None
        self.rocs_dict = None
        self.model_names = [m.__class__.__name__ for m in models]
        self.weights = []
        
    def fit_all(self, X_train, y_train):
        for m in self.models:
            print("Fitting", m.__class__.__name__)
            m.fit(X_train, y_train)
            print(m.__class__.__name__, 'fit.')
        
    
    def evaluate_all(self, X_test, y_true, metric = 'acc'):
        accs = [accuracy_score(y_true, m.predict(X_test)) for m in self.models]
        accs_dict = dict(zip(self.model_names, accs))
        
        
        rocs = [roc_auc_score(y_true, m.predict(X_test)) for m in self.models]
        rocs_dict = dict(zip(self.model_names, rocs))
        
        self.rocs_dict = rocs_dict
        self.accs_dict = accs_dict
        
        if metric == 'acc':
            return accs_dict
        
        return rocs_dict
        
    def get_weights(self):
        return self.rocs_dict
    
    def get_model_names(self):
        return self.model_names
        
        
        
        

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['Avg_Presence']),
                                                   df['Avg_Presence'], random_state = 73)

In [ ]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import *


# Here is the ensemble of models - again, I think the tree methods are far stronger
mlp = MLPClassifier(max_iter = 1000, random_state = 73)
logit = LogisticRegression(max_iter = 10000)
rf = RandomForestClassifier()
brt = GradientBoostingClassifier()
dt = DecisionTreeClassifier()

# Construct ensemble object
ensemble = Ensemble([mlp, brt, dt, rf, logit]) 
ensemble.fit_all(X_train,y_train)

print(ensemble.evaluate_all(X_test, y_test, metric = 'roc'))
#y_pred = ensemble.evaluate_ensemble(X_test, y_test)

#accuracy_score(y_test,y_pred)

In [ ]:
## This chunk of code builds the ensemble 
from sklearn.ensemble import VotingClassifier

# This stores the weights that each model should have when voting
weights = [ensemble.get_weights()[c] for c in ensemble.get_model_names()]

# Here is where we might turn off different models due to lower accuracy score. 
# I don't know how the models will predict EBT
# Might Turn off ANN
weights[2] = 0

#Also turn off Logistic Regression and Decision tree
# weights[1] = 0
# weights[-1] = 0

vc_names = [('RF', rf), ('Logit', logit),
('ANN', mlp), ('BRT', brt), ('DT', dt)]

vc = VotingClassifier(estimators=vc_names, voting='soft', weights = weights)
vc.fit(X_train, y_train)
print(accuracy_score(y_test, vc.predict(X_test)))

In [ ]:
# Check out confusion matrix
confusion_matrix(y_test, vc.predict(X_test))

In [ ]:
# Project Model to Environmental Space
#Here, we are interested in the "pretty map" habitat suitability indexes.
first_decade = pd.read_csv(decade1, index_col = 'huc12')
drops = [c for c in first_decade if c not in X_train.columns]
bad_hucs = first_decade.index[first_decade.Max_LST_Annual.isna()]
first_decade.drop(bad_hucs,inplace=True)
first_decade.drop(columns = drops, inplace=True)


#second_decade = pd.read_csv(decade2 ,index_col='huc12')
#second_decade.drop(bad_hucs,inplace=True)
#second_decade.drop(columns = drops, inplace=True)


In [ ]:
def MESS(train_df, pred_df):
#     Let min_i be the minimum value of variable V_i
#     over the reference point set, and similarly for max_i.
    mins = dict(X_train.min())
    maxs = dict(X_train.max())
    
    def calculate_s(column):
        # Let f_i be the percent of reference points 
        # whose value of variable V_i is smaller than p_i.

        # First store training values
        values = train_df[column]

        # Find f_i as above
        sims = []
        for element in np.array(pred_df[column]):
            f = np.count_nonzero((values < element))/values.size

            # Find Similarity:
            if f == 0:
                sim = ((element - mins[column]) / (maxs[column] - mins[column])) * 100

            elif ((f > 0) & (f <= 50)):
                sim = 2 * f

            elif ((f > 50) & (f < 100)):
                sim = 2 * (100-f)

            elif f == 100:
                sim = ((maxs[column] - element) / (maxs[column] - mins[column])) * 100

            sims.append(sim)


        return sims
    
    # Embedd Dataframe with sim values
    sim_df = pd.DataFrame()
    for c in pred_df.columns:
        sim_df[c] = calculate_s(c)
    
    
    minimum_similarity = sim_df.min(axis=1)
    
    return minimum_similarity

In [ ]:
## GET PREDICTION UNCERTAINTY
predictions_first = []
#predictions_second = []
for ind,model in enumerate(vc.estimators_):
    # Don't use Logistic Regression or MLP
    if ind in [1,2]:
        continue

    predict_prob = [a[0] for a in model.predict_proba(first_decade)]
    predictions_first.append(predict_prob)
    
    #predict_prob = [a[0] for a in model.predict_proba(second_decade)]
    #predictions_second.append(predict_prob)

## \GET PREDICTION UNCERTAINTY
    
first_decade_pred = pd.DataFrame({'huc12':pd.Series(first_decade.index),
                                  'prediction_ensemble':vc.predict(first_decade),
                                  'prediction_proba':[a[0] for a in vc.predict_proba(first_decade)],
                                  'prediction_uncertainty': np.std(np.array(predictions_first), axis=0),
                                 'MESS': MESS(X_train,first_decade)})
#second_decade_pred = pd.DataFrame({'huc12':pd.Series(second_decade.index),
                                  #'prediction_ensemble':vc.predict(second_decade),
                                  #'prediction_proba':[a[0] for a in vc.predict_proba(second_decade)],
                                  #'prediction_uncertainty': np.std(np.array(predictions_second), axis=0),
                                  #'MESS': MESS(X_train,second_decade)})

In [ ]:
#Merge these predictions with the Shape File, which needs to be downloaded from the USGS website
# This needs to be downloaded from the USGS website
hucs = gpd.read_file('/Users/myles/Documents/flbs/MT_HUCS.geojson')
hucs['huc12']=hucs['huc12'].astype('int64')

#drops = ['MESS_mean', 'Most_Dissimilar_Variable_majority',
       #'FIRST_Decade_QC_mean', 'Second_Decade_QC_mean', 'FIrst_Decade_QC_sum',
       #'FIRST_DECADE_QC_sum', 'FIRST_DECADE_QC_max', 'SECOND_DECADE_QCmax']
#hucs.drop(columns=drops,inplace=True)

hucs_pred_first = hucs.merge(first_decade_pred,on='huc12')
#hucs_pred_second = hucs.merge(second_decade_pred,on='huc12')

In [ ]:
decade_1 = pd.DataFrame(hucs_pred_first)

In [ ]:
decade_1.to_csv(decade1_pred, index=False)

In [ ]:
#decade_2 = pd.DataFrame(hucs_pred_second)

In [ ]:
#decade_2.to_csv(decade2_pred, index=False)

In [ ]:
# Get Feature Importances

In [ ]:
This will be the backbone of the Community-level "top predictors" analysis.

In [ ]:
# Function that runs the "Drop Column" Feature importance technique 
# I actually have these in a separate .py file which would be much cleaner. 
from sklearn.base import clone

# Short function to create sorted data frame of feature importances
import pandas as pd
def make_imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

def drop_col(model, X_train, y_train, random_state = 42):
    #Clone the model
    model_clone = clone(model)
    #Reset random state
    model_clone.random_state = random_state
    #Train and score the benchmark model
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)
    #Store importances
    importances = []
    
    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col,axis=1),y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis = 1), y_train)
        importances.append(benchmark_score - drop_col_score)
        
    importances_df = make_imp_df(X_train.columns, importances)
    return importances_df

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE, SelectFromModel


In [ ]:
# Do a bit of manipulation and run all these feature importance techniques
vc_names = [('RF', rf), ('Logit', logit), ('ANN', mlp), ('BRT', brt), ('DT', dt)]
def make_dict():
    return dict(zip([tup[0] for tup in vc_names], [None]))


rfe_dict = make_dict()
perm_dict = make_dict()
drop_dict = make_dict()

for alg in vc.named_estimators:
    dict_name = alg
    clf = vc.named_estimators[dict_name]
    
    if dict_name == 'ANN':
        continue
    
    print("Considering", clf)
    
    # Find Recursive feature elimination for each classifier
    rfe_selector = RFE(estimator=clf, n_features_to_select=3, step=1, verbose=5)
    rfe_selector.fit(X_train,y_train)
    rfe_support = rfe_selector.get_support()
    rfe_features = X_train.loc[:,rfe_support].columns.tolist()
    
    # Add to rfe_dict
    rfe_dict[dict_name] = rfe_features
    
    
    # //========================================================
    # Find Permuation importance for each classifier
    perm_imp = permutation_importance(clf, X_train, y_train,
                          n_repeats=30,
                          random_state = 0)
    
    perm_imp['feature'] = X_train.columns
    perm_features = pd.DataFrame(perm_imp['feature'],perm_imp['importances_mean'],columns = ['Feature']) \
                            .sort_index(ascending=False)['Feature'].values[:3]
    
    # Add permutation features to dict
    perm_dict[dict_name] = perm_features
    
    
    
    #//========================================================
    # Find Drop Columns importance for each classifier
    drop_col_feats = drop_col(clf, X_train, y_train, random_state = 10)
    drop_col_three = drop_col_feats.sort_values('feature_importance',ascending = False)['feature'][:3]
    
    drop_dict[dict_name] = drop_col_three
    
    
print("Done")




In [ ]:
def mark_true(series):
    return [True if feature in series else False for feature in X_train.columns]

def rename_dict(dictionary, tek_name):
    return_names = []
    return_lists = []
    
    for item in dictionary.items():
        return_names.append(tek_name + str(item[0]))
        return_lists.append(mark_true(list(item[1])))
        
    return dict(zip(return_names, return_lists))
        

In [ ]:
#We end up with a dataframe that says, for any model / feature importance technique, whether or not a feature ended up in the top N (i.e. whether or not that feature is important).
features_df = pd.concat([pd.DataFrame(rename_dict(perm_dict,'PERM_')).reset_index(drop=True),
                        pd.DataFrame(rename_dict(rfe_dict,'RFE_')),
                        pd.DataFrame(rename_dict(drop_dict, 'DROP_'))],axis=1)


features_df['Feature'] = X_train.columns
features_df['Total'] = np.sum(features_df, axis=1)
features_df.sort_values(['Total','Feature'] , ascending=False,inplace=True)
features_df

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8,4))
plt.setp( ax.xaxis.get_majorticklabels(), rotation=-45, ha="left" )
plt.setp( ax.xaxis.get_majorticklabels(), rotation=-45, ha="left" )

# plt.xticks(rotation=75)
plt.bar(features_df['Feature'], features_df['Total'])
plt.title("Variable importances")
plt.ylabel("Number of times a variable appeared in top 3")


In [ ]:
That's it. These are the results we are looking for regarding the NAS community-level study.